In [ ]:
from datasets import load_dataset


In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
def formatting_func(example):
    
    if example['Speaker'] == "Any":
        npc_type = "any type of NPC"
    else:
        npc_type = f'the NPC type {example["Speaker"]}'
    
    if example['Event'] == "Greeting":
        
        return f"""In the world of Skyrim from the game Elder Scrolls V, in the case of a {example['Event']}, between an NPC and the Player character, {npc_type} greets the player with the following dialogue: {example['Dialogue']}, in a {example['Tone']} tone."""
    
    
    if example['Event'] == "Goodbye":
        
        return f"""In the world of Skyrim from the game Elder Scrolls V, in the case of a {example['Event']}, between an NPC and the Player character,{npc_type} tells their goodbyes to the Player with the following dialogue: {example['Dialogue']} """
    
    
    if example['Event'] == "NPCs fighting over a player's dropped item":
        
        return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of {example['Event']}, there can be up to 5 different NPCs arguing over an item
        that the player character dropped and 2 bystander NPCs. Up to five lines of dialogue can be exchanged between up to five different types of NPCs, meaning that the conversation ends after 5 lines of dialogue have been said.
        
        In this case one of the NPCs that takes part in the arguing and is {npc_type} says the following line: {example['Dialogue']} as the
        {example['Line_of_dialogue']} line of the dialogue."""
    
    if example['Event'] == "NPC asking for the Player's dropped armor":
        
        if example['Speaker'] == "Player":
        
            return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of an {example['Event']} there is a conversation between the Player character and an NPC about the player's dropped item. 
            
            In this case the Player character says the following line: {example['Dialogue']} in response to the NPC's previous line of dialogue: {example['Response_to']}."""
        
        if example['Response_to'] == "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of {example['Event']} there is a conversation between the Player character and an NPC about the player's dropped item. 
            
            In this case {npc_type} says the following line: {example['Dialogue']} in response to the Player character's previous line of dialogue: {example['Response_to']}."""
        else:
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of {example['Event']} there is a conversation between the Player character and an NPC about the player's dropped item. 
            
            In this case {npc_type} says the following line: {example['Dialogue']} to initiate a conversation about the Player's dropped item."""
    
    if example['Condition'] == "in combat":
        
        return f"""In the world of Skyrim from the game Elder Scrolls V, when the Player character and a friendly NPC are {example['Condition']}, {npc_type} uses this line of dialogue: {example['Dialogue']}, when {example['Event']}"""
        
    if example['Result'] is not None:
        
            return f"""In the world of Skyrim from the game Elder Scrolls V, when the Player character tries to check if an NPC of any type can be persuaded, the NPC uses the following line of dialogue: {example['Dialogue']} when {example['Event']} and as a result {example['Result']} """
    
    if example['Event'] == "an NPC spots the Player character having an amulet of mara":
        
        if example['Speaker'] == "Player":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, when {example['Event']} an interaction begins between the two about the possibility of marriage. 
            
            In this case the Player character says the following line: {example['Dialogue']} as a response to the the NPC's dialogue line:{example['Response_to']}"""
        
        if example['Response_to'] == "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, when {example['Event']} an interaction begins between the two about the possibility of marriage.
                
            In this case {npc_type} says the following line: {example['Dialogue']} to initiate the conversation about marriage."""
        
        else:
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, when {example['Event']} an interaction begins between the two about the possibility of marriage. 
            
            In this case {npc_type} says the following line: {example['Dialogue']} as a response to the the Player's dialogue line:{example['Response_to']}"""
    
    if example['Race'] is not None:
        
        if example['Condition'] != "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special reactions to the race of the Player character, so in the case of {example['Event']} if the Player's race is: {example['Race']} the Guard reacts with this line of dialogue: {example['Dialogue']} on the condition that the {example['Condition']}. """
        
        else:
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special reactions to the race of the Player character, so in the case of {example['Event']} if the Player's race is: {example['Race']} the Guard reacts with this line of dialogue: {example['Dialogue']}. """
     
    if example['Equipment'] is not None:
        
        if example['Condition'] != "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special reactions to the equipment the player is holding or has equipped, so when in the case of a {example['Event']}, the Guard uses this line of dialogue:{example['Dialogue']} on the condition that {example['Condition']}."""
        
        else: 
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special reactions to the equipment the player is holding or has equipped, so when in the case of a {example['Event']}, the Guard uses this line of dialogue:{example['Dialogue']}."""
            
    if example['Location'] is not None:
        
        if example['Condition'] != "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special interactions with the Player character that are location based, meaning that the Guards sometimes interact with the Player using town specific dialogue.

            In this case a Guard in the town of {example['Location']} might use this line of dialogue {example['Dialogue']} when the Player interacts with them, on the condition that {example['Condition']}."""
        
        else: 
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, Town Guards have special interactions with the Player character that are location based, meaning that the Guards sometimes interact with the Player using town specific dialogue.

            In this case a Guard in the town of {example['Location']} might use this line of dialogue {example['Dialogue']} when the Player interacts with them."""
        
    if example['Condition'] is not None:
        
        if example['Condition'] != "nan":
            
            return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of {example['Event']}, {npc_type} says the following line to the Player ch+aracter: {example['Dialogue']} on the condition that {example['Condition']}."""
    
    return f"""In the world of Skyrim from the game Elder Scrolls V, in the event of {example['Event']}, {npc_type} says the following line to the Player character: {example['Dialogue']}."""

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token


def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

In [ ]:
import matplotlib.pyplot as plt


def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset['train']]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset['train']]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()


plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [ ]:
max_length = 120  # This was an appropriate max length for my dataset


def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt2)

In [ ]:
print(tokenized_train_dataset['train'][1]['input_ids'])

In [ ]:
plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [ ]:
eval_prompt = ("Start a conversation where Dutch van der Linde plans a new heist Stealing the USA gold reserves")

In [ ]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0],
                                skip_special_tokens=True))

In [ ]:
from peft import PeftConfig

adapter_model_id = "ybelkada/opt-350m-lora"
peft_config = PeftConfig.from_pretrained(adapter_model_id)
# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()

In [ ]:
print("###Character: " + test_dataset['train'][200]['character'] + "\n  responded with: " + test_dataset['train'][200][
    'dialogue'] + "\n to what was said by: " + test_dataset['train'][200]['response_to'])

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
print(model)

In [ ]:
if torch.cuda.device_count() > 1:  # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
model = accelerator.prepare_model(model)

In [ ]:
import transformers
from datetime import datetime

project = "own-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_val_dataset["train"],
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=200,
        learning_rate=2.5e-5,  # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,  # When to start reporting loss
        logging_dir="./logs",  # Directory for storing logs
        save_strategy="steps",  # Save the model checkpoint every logging step
        save_steps=25,  # Save checkpoints every 50 steps
        evaluation_strategy="steps",  # Evaluate the model every logging step
        eval_steps=25,  # Evaluate and save checkpoints every 50 steps
        do_eval=True,
        # Perform evaluation at the end of training         # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"  # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-own-finetune/checkpoint-50")

In [ ]:
eval_prompt = "Start a conversation where Dutch van der Linde plans a new heist"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=280, repetition_penalty=1.15)[0],
                                skip_special_tokens=True))

In [ ]:
eval_prompt = "Imagine a scenario in Red Dead Redemption 2 where Abe informs John Marston that there is a courier outside of his ranch waiting to deliver him a telegram. John Marston is intrigued because he was not expecting one and makes a jokes about the contents of the telegram and then follows abe to the courier, finally he asks if the courier has a telegram for him. Generate the dialogue John and Abe during this scenario."
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=162, repetition_penalty=1.15)[0],
                                skip_special_tokens=True))

In [ ]:
eval_prompt = "Imagine a scenario in Red Dead Redemption 2 where Abe informs John Marston that there is a courier outside of his ranch waiting to deliver him a telegram. John Marston is intrigued because he was not expecting one and makes a jokes about the contents of the telegram and then follows abe to the courier, finally he asks if the courier has a telegram for him. Generate the dialogue John and Abe during this scenario."
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=170, repetition_penalty=1.15)[0],
                                skip_special_tokens=True))

In [ ]:
eval_prompt = """

[INST]Continue the last 2 lines of conversation between Arthur Morgan and a doctor in Red Dead Redemption 2 and not any further [/INST]

Doctor: Sure, thank you. Now, what’s wrong? I mean, what appear to be the symptoms?

Arthur Morgan: Well, I think you’ve heard them… I’m, I’m coughing.

Doctor: Is there any blood?

Arthur Morgan: Sometimes.

Doctor: Okay now, here… breathe. Again. Let me see your tongue. Now say ahh.

Arthur Morgan: Aah… What is it?

Doctor: It’s not good news.

Arthur Morgan: Well I guessed that.

Doctor: """

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=43, repetition_penalty=1.15)[0],
                                skip_special_tokens=True))

# Rouge Score

In [ ]:
generated_dialogues = ["Doctor:  I'm afraid you have tuberculosis. We need to start treatment right away.",
                       "Arthur Morgan: Tuberculosis... That's a death sentence isn't it?"]

reference_dialogues = ["Doctor: You got tuberculosis. I’m really sorry for you, son, it’s a hell of a thing.",
                       "Arthur Morgan: Aah… What is it?"]

print(len(reference_dialogues))

from rouge import Rouge

rouge = Rouge()
rouge_scores = rouge.get_scores(' '.join(generated_dialogues), ' '.join(reference_dialogues[0]))

# Output the scores 

print(f"ROUGE: {rouge_scores}")


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

# Tokenize the reference and generated dialogues into words
reference_dialogues = [
    ["Doctor:", "You", "got", "tuberculosis.", "I’m", "really", "sorry", "for", "you,", "son,", "it’s", "a", "hell", "of", "a", "thing."],
    ["Arthur", "Morgan:", "Aah…", "What", "is", "it?"]
]
generated_dialogues = [
    ["Doctor:", "I'm", "afraid", "you", "have", "tuberculosis.", "We", "need", "to", "start", "treatment", "right", "away."],
    ["Arthur", "Morgan:", "Tuberculosis...", "That's", "a", "death", "sentence", "isn't", "it?"]
]


# Calculate BLEU score
bleu_score = sentence_bleu(reference_dialogues, generated_dialogues)
print(f"BLEU: {bleu_score}")